## Fairness Metrics on words

In [2]:
# Importing the required libraries
import numpy as np
import pandas as pd
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer 

from sklearn.model_selection import train_test_split
import re
import pickle
import warnings
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import mutual_info_classif

import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# sys.path.insert(0, parentdir)
# import model
from model.load_model import load_model, predict
model = load_model()


2024-06-08 23:33:23.686999: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-08 23:33:23.798868: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 23:33:23.798933: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 23:33:23.798970: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-08 23:33:23.819594: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

In [2]:
# model = load_model()
# predict("text", model)
# # predict("dupa", model, tokenizer, device)

OSError: No such file or directory: 'Decision Tree'

In [3]:
# loading dataset
# importing the dataset
first_data = pd.read_csv('../data/fake_and_real_data.csv')
first_data.head()
second_data = pd.read_csv('../data/WELFake_Dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/fake_and_real_data.csv'

In [ ]:
# Checking for missing values

second_data.isnull().sum()

# Dropping the missing values
second_data = second_data.dropna()
# Checking for duplicates

second_data.duplicated().sum()

# Drop unnamed column
second_data.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# concatinating title and text
second_data['text'] = second_data['title'] + ' ' + second_data['text']
# Dropping the title column
second_data = second_data.drop(['title'], axis=1)



In [ ]:
first_data.drop_duplicates(inplace=True)


first_data['label_t'] = 0


first_data.head()

In [ ]:
# Preprocessing the data
first_data['label_t'] = (first_data['label'] != 'Fake').astype(int)
first_data = first_data.drop(['label'], axis=1)
first_data = first_data.rename(columns={'label_t': 'label'})
first_data = first_data.rename(columns={'Text': 'text'})
first_data.head()


In [ ]:
# Dropping the unnecessary columns
first_data.head()
# converting the text to lowercase
def preprocess(text):
    text = re.sub(r'[^\w\s\']', ' ', text)
    text = re.sub(r' +', ' ', text)
    return text.strip().lower()

first_data['text'] = first_data['text'].apply(preprocess)
second_data['text'] = second_data['text'].apply(preprocess)






In [ ]:
first_data.head()
second_data.head()

concat_data = pd.concat([first_data, second_data], axis=0)
concat_data.head()


In [ ]:
# is trump inclouded 

concat_data['does_contain_trump'] = concat_data['text'].apply(lambda x: 'trump' in x)

In [ ]:
concat_data.head()

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer 

# # model.predict(concat_data["text"][None, :].numpy())
# # import torch
# # input_tensor = torch.tensor(concat_data.values, dtype=torch.float32)
# vect = TfidfVectorizer(stop_words='english', max_df=0.7)
# xv_train = vect.fit_transform(concat_data["text"])
# model.predict(xv_train)

In [ ]:
concat_data['does_contain_trump'].value_counts()



In [ ]:
# y_pred = model.predict(concat_data["text"])

# y_pred

In [ ]:

# vect = TfidfVectorizer(stop_words='english', max_df=0.7)
# data = vect.transform(concat_data["text"])
# # xv_test = vect.transform(x_test)
# model.predict(data)
model.predict([[5]])



In [ ]:
def fairness_metrics(df):
    """Calculate fairness for subgroup of population"""
    
    #Confusion Matrix
    cm=confusion_matrix(df['y'],df['y_pred'])
    TN, FP, FN, TP = cm.ravel()
    
    N = TP+FP+FN+TN #Total population
    ACC = (TP+TN)/N #Accuracy
    TPR = TP/(TP+FN) # True positive rate
    FPR = FP/(FP+TN) # False positive rate
    FNR = FN/(TP+FN) # False negative rate
    PPP = (TP + FP)/N # % predicted as positive
    
    return np.array([ACC, TPR, FPR, FNR, PPP])

In [ ]:
# trump_unprivileged_group = [{'does_contain_trump': 0}] 
# trump_unprivileged_group

# Fairness with all values 

In [ ]:
# import mlflow
# import os

# # Define the run ID and artifact path
# run_id = os.environ.get('RUN_ID')
# # artifact_path = mlflow.get_artifact_uri(run_id)

# # Print the artifact path to locate the MLmodel file
# # print("Artifact path:", artifact_path)
# # loaded_model = mlflow.pytorch.load_model(artifact_path)
# model_path = mlflow.artifacts.download_artifacts(run_id=run_id)
# print(model_path)

In [ ]:
concat_data["y"] = [1 if y == 'Real' else 0 for y in concat_data['label']] 
concat_data["y_pred"] = [model.predict(y) for y in concat_data['text']]
result = fairness_metrics(concat_data)
result

# Fairness when having trump 

In [ ]:
df_containing_trump = concat_data.loc[concat_data['does_contain_trump'] == True]
result = fairness_metrics(df_containing_trump)
result
# df_containing_trump
# df["does_contain_trump"]
# df

In [ ]:
df_without_trump = concat_data.loc[concat_data['does_contain_trump'] == False]
result = fairness_metrics(df_without_trump)
result

In [ ]:
# # Simply import it from sklearn
# from sklearn.model_selection import train_test_split

# dataset = concat_data 

# # Separating features from target feature
# features = dataset.columns.tolist()
# features.remove('label')
# target = 'label'
# X = dataset[features]
# y = dataset[target]

# # Define four sets and apply the function
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                     test_size=0.2, # 0.2 indicates a test set size of 20%
#                                                     random_state=42)

In [ ]:
# ## applying fairness tests
# from sklearn.feature_extraction.text import TfidfVectorizer 
# # Import the classifier and the metrics from sklearn
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import accuracy_score, recall_score, f1_score

# dt_clf = DecisionTreeClassifier(random_state=42)
# vect = TfidfVectorizer(stop_words='english', max_df=0.7)
# xv_train = vect.fit_transform(X_train['text'])
# xv_test = vect.transform(X_test['text'])


# print(f"Value to test if testing started")
# # The fit function will do the trick
# dt_clf.fit(xv_train, y_train)

# print(f"Value to test if testing ended")
# # After the training phase, the model will be tested by predicting the values on the test set
# dt_predictions = dt_clf.predict(xv_test)

# dt_accuracy = accuracy_score(y_test, dt_predictions)
# dt_recall = recall_score(y_test, dt_predictions)
# dt_f1_score = f1_score(y_test, dt_predictions)
# print(f"Decision Tree Accuracy: {dt_accuracy}")
# print(f"Decision Tree Recall: {dt_recall}")
# print(f"Decision Tree F1 Score: {dt_f1_score}")

In [ ]:

# from aif360.datasets import BinaryLabelDataset
# from aif360.datasets import StandardDataset
# from aif360.metrics import ClassificationMetric


# # We want to check the fairness level regarding the protected attribute "sex"
# trump_features = ['does_contain_trump']


# dataset = xv_test.toarray()
# dataset['label'] = y_test  # and join the target feature with the others

# predictions = dataset.copy(deep=True)  # we do the same task
# # but this time the target feature is made by the predictions of our model
# predictions['label'] = dt_predictions

# # In this way, we have two datasets. One (dataset) is the original test set containing the original values of features,
# # the other (predictions) contains the original values except for the target one, that is now made of model's predictions

# # These will be used by AIF to compare the classifications of the model with the original values to
# # understand if the model's answers create favouritism toward the privileged attribute


# # This is the object made of the original dataset
# aif_sex_dataset = BinaryLabelDataset(  # Base class for all structured datasets with binary labels.
#     df=dataset,
#     # This means that a prediction is biased toward the privileged attribute if its value is 1 (True)
#     favorable_label=1,
#     unfavorable_label=0,
#     label_names=[target],
#     protected_attribute_names=trump_features,
#     # here we tell AIF that we want to check for predictions
#     privileged_protected_attributes=['does_contain_trump'],
#     # that somehow privilege the attribute "sex_Male"
# )

# # We do the same thing but with the predictions dataset
# aif_sex_pred = BinaryLabelDataset(
#     df=predictions,
#     favorable_label=1,
#     unfavorable_label=0,
#     label_names=[target],
#     protected_attribute_names=trump_features,
#     privileged_protected_attributes=['does_contain_trump'],
# )

# trump_privileged_group = [{'does_contain_trump': 1}]
# trump_unprivileged_group = [{'does_contain_trump': 0}]    

# # We provide the ClassificationMetric object with all the information needed:
# # aif_sex_dataset - The original test set
# # aif_sex_pred - A dataset containing the predictions of the model
# # sex_privileged_group - The privileged group
# # sex_unprivileged_group - The unprivileged group
# fairness_metrics = ClassificationMetric(dataset=aif_sex_dataset,
#                                         classified_dataset=aif_sex_pred,
#                                         unprivileged_groups=trump_unprivileged_group,
#                                         privileged_groups=trump_unprivileged_group)

# # Values less than 0 indicate that privileged group has higher
# # proportion of predicted positive outcomes than unprivileged group.
# # Value higher than 0 indicates that unprivileged group has higher proportion
# # of predicted positive outcomes than privileged group.
# SPD = round(fairness_metrics.statistical_parity_difference(), 3)

# # Measures the deviation from the equality of opportunity, which means that the same
# # proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
# EOD = round(fairness_metrics.equal_opportunity_difference(), 3)

# # Average of difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
# # A value of 0 indicates equality of odds, which means that samples in both the privileged and unprivileged
# # groups have the same probability of being classified positively.
# AOD = round(fairness_metrics.average_odds_difference(), 3)

# print(f"Statistical Parity Difference (SPD): {SPD}")
# print(f"Equal Opportunity Difference (EOD): {EOD}")
# print(f"Average Odds Difference: {AOD}")

In [ ]:
# import pandas as pd
# import warnings
# from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE

# warnings.filterwarnings("ignore")
# df = pd.read_csv('WELFake_Dataset.csv')

# # bool_cols = df.select_dtypes(include=['bool']).columns
# # df[bool_cols] = df[bool_cols].astype(int)

# X = df.drop(columns=['label'])
# y = df['label']

# smote = SMOTE(random_state=42)
# X_smote, y_smote = smote.fit_resample(X, y)

# X = X_smote
# y = y_smote

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# import xgboost as xgb

# from sklearn.metrics import accuracy_score,confusion_matrix
# from sklearn.metrics import classification_report
# from sklearn.feature_selection import mutual_info_classif

# import sys
# sys.path.insert(0, '..')
# from load_model import load_model, predict
# sys.path

In [ ]:


# df = pd.read_csv('fake_and_real_data.csv')
# df["y"] = [1 if y == 'Real' else 0 for y in df['label']] 
# df["y_pred"] = ["Real" for y in df['Text']]
# df

In [ ]:
# link_to_source = "https://towardsdatascience.com/analysing-fairness-in-machine-learning-with-python-96a9ab0d0705"
# df = pd.read_csv('fake_and_real_data.csv')
# model, tokenizer, device = load_model()

# df_fair = df[['label']]
# df["y"] = [1 if y == 'Real' else 0 for y in df['y']] 
# df["y_pred"] = [model.predict(y, model, tokenizer, device) for y in df['Text']]
# counts = df['text'].value_counts()
# labels = counts.index

# #Plot pie chart
# plt.pie(counts, startangle=90)
# plt.legend(labels, loc=2,fontsize=15)
# plt.title("Truthness of news",size=20)
# result = fairness_metrics(df)

# def fairness_metrics(df):
#     """Calculate fairness for subgroup of population"""
    
#     #Confusion Matrix
#     cm=confusion_matrix(df['y'],df['y_pred'])
#     TN, FP, FN, TP = cm.ravel()
    
#     N = TP+FP+FN+TN #Total population
#     ACC = (TP+TN)/N #Accuracy
#     TPR = TP/(TP+FN) # True positive rate
#     FPR = FP/(FP+TN) # False positive rate
#     FNR = FN/(TP+FN) # False negative rate
#     PPP = (TP + FP)/N # % predicted as positive
    
#     return np.array([ACC, TPR, FPR, FNR, PPP])